# Tensores

Basado en los capítulos 1 y 2 de **Learning TensorFlow**, de Tom Hope, Yehezkel S. Resheff & Itay Lieder, y los capítulos 1 y 2 de **Deep Learning with Python**, de Francois Chollet, Manning, 2017

In [ ]:
import numpy as np

Definición de tensores de rango 0, 1, y 2. Se trata de matrices multidimensionales que se definen como numpy arrays.

In [ ]:
x = np.array(12)
y = np.array([12,3,6,14])
z = np.array([[5, 78, 2, 34, 0],
              [6, 79, 3, 35, 1],
              [7, 80, 4, 36, 2]])
print(x, '\n', y, '\n', z)

Obtener el dimensionamiento (el número de ejes). Los escalares tienen 0 dimensiones, por convención.

In [ ]:
print(x.ndim, y.ndim, z.ndim)

Obtener la forma de los tensores

In [ ]:
print(x.shape, y.shape, z.shape)

Se puede dar el caso de que se abuse utilizando el término "dimensiones" para referirse al "rango" o al número de "ejes" de un tensor.

El primer tensor es un escalar. el segundo es un array con un eje de 4 dimensiones. El tercero es un tensor de rango 2, con 3 dimensiones en el primer eje y 5 dimensiones en el segundo eje.

Tipo de datos de los tensores

In [ ]:
print(x.dtype, y.dtype, z.dtype)

# TensorFlow

In [ ]:
import tensorflow as tf

Vamos a analizar unas operaciones simples que quedarán plasmadas en el dataflow graph. Lo primero que hacemos es limpiar el grafo por defecto de TensorFlow.

In [ ]:
tf.reset_default_graph()

Inicialmente, para poder interactuar directamente con TensorFlow sin necesidad de una sesión, configuramos una sesión interactiva:

In [ ]:
tf.InteractiveSession()

## Operaciones básicas

Primero mostremos que se pueden crear tensores de valores constantes

In [ ]:
c=tf.constant(2, name="constante_2")

Cada tensor es en realidad una referencia a un objeto tensor que contiene la información. Así hayamos creado una referencia llamada **"c"** que apunta al tensor, esta referencia existe solo en Python, mas no en el ambiente de ejecución nativo donde en realidad está la sesión actual y la estructura correspondiente. Si queremos referenos a un tensor nativamente, hay que especificar su nombre con el parámetro **name**.

Para poder acceder a su contenido es necesario evaluarlo (solo funciona de esta manera cuando se está en una sesión interactiva):

In [ ]:
c.eval()

De hecho, antes de evaluar **c** en la sesión interactiva, solo se había específicado las operaciones que se podían ejecutar, sin haberlas ejecutado (en este caso, la declaración de una constante con valor 2). Al programar en TensorFlow solo se especifica lo que se quiere hacer, la idea es poder definir todas las operaciones declarativamente antes de ejecutarlas dentro de una sesión, minimizando el costo de pasar del ambiente interpretado de Python al ambiente nativo de bajo nivel.

TensorFlow consta de varios métodos que permiten crear y manipular tensores. Por ejemplo, creemos un tensor de rango 2, con 5 filas y 3 columnas lleno de "1"s.

In [ ]:
o = tf.ones((3,2), name="unos")
o.eval()

Podemos crear tensores con cualquier valor definido o con valores aleatorios:

In [ ]:
b = tf.fill((2,5), 5.0, name="cincos-b")
b.eval()

In [ ]:
r = tf.random_normal((2,5), mean=5, stddev=1, seed=1234, name="aleatorio-r")
r.eval()

TensorFlow tiene que redefinir todas las operaciones con matrices que se encuentran en numpy de manera nativa, pero se sobrecargan los operadores básicos en Python. Por ejemplo:

In [ ]:
suma = b + r
suma.eval()

In [ ]:
suma = tf.add(b, r, name="suma")
suma.eval()

Cada vez que se evalua un tensor random se obtienen valores diferentes 

Visualicemos el dataflow graph utilizando un servicio web de TensorBoard (código copiado de: https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/tutorials/deepdream/deepdream.ipynb)

In [ ]:
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [ ]:
show_graph(tf.get_default_graph().as_graph_def())

La operación de **broadcasting** permite no tener que definir estructuras con las dimensiones competas; TensorFlow se ncarga de completarlas con una copia dadas las dimensiones especificadas

In [ ]:
f = 2 * b
f.eval()

In [ ]:
a =tf.ones((2,2), name="matriz_de_unos")
a.eval()

In [ ]:
b = tf.range(0, 2, 1, dtype=tf.float32, name="array-rango")
b.eval()

In [ ]:
c = a+b
c.eval()

Para renombrar un nodo (operación) en TF se usa el método tf.identity:

In [ ]:
tf.identity(c, name="suma_matriz_array")

In [ ]:
show_graph(tf.get_default_graph().as_graph_def())

Como con numpy, también es común tener que hacer reorganizaciones de tensores para que las dimensiones concuerden: 

Crear una matriz identidad y un array para ilustrar un ejemplo de broadcasting al sumar la matriz con el array

In [ ]:
a = tf.eye(5)
a.eval()

In [ ]:
b = tf.range(0,5, 1, dtype = tf.float32)
b.eval()

In [ ]:
c= a+b
c.eval()

Se puede transponer la matriz:

In [ ]:
d = tf.transpose(c)
d.eval()

In [ ]:
d.shape

Se puede crear un array con un valor específico (en este caso 2). El punto sirve para darle implícitamente el tipo float al tensor:

In [ ]:
e = tf.fill((5,1), 2.)
e.eval()

In [ ]:
e.shape

Multiplicación de la matriz (d) de 5x5 por el array de 5x1 (e)

In [ ]:
f= tf.matmul(d, e)
f.eval()

Ahora vamos a ilustrar las operaciones de reorganización dimensional de tensores:
- Empezamos con el array *g* (tensor de rango 1) con 12 dimensiones
- lo convertimos en un tensor *h* de rango 2, con dimensiones 1 (filas) y 12 (columnas)
- lo convertimos en un tensor *i* de rango 2, con dimensiones 3 (filas) y 4 (columnas)
- lo convertimos en un tensor *j* de rango 3, con dimensiones 3 (profundidad), 2 (filas) y 2 (columnas)

In [ ]:
f = tf.range(1,13,1)
f.eval()

In [ ]:
f.shape

In [ ]:
g = tf.reshape(f, (1, 12))
g.eval()

In [ ]:
g.shape

In [ ]:
h = tf.reshape(g, (2, 6))
h.eval()

In [ ]:
i = tf.reshape(h, (3, 4))
i.eval()

In [ ]:
j = tf.reshape(h, (3, 2, 2))
j.eval()

Al igual que con numpy.expand y numpy.squeeze, podemos ampliar el rango de los tensores y aplanarlos, respectivamente:

In [ ]:
g = tf.expand_dims(g, 1)
j.eval()

In [ ]:
k=tf.squeeze(j)
k.eval()

## Ejemplo de definición y ejecución de un computation graph en una sesión

### Definición del grafo

Como ejemplo tomemos el dataset de MNIST que tiene como inputs imágenes de 28x28 pixeles en escala de grises.
Aplanando las imágenes, tenemos n0=784 inputs (neuronas de entrada). 
Imaginen ahora que hay una capa de n1=100 neuronas de procesamiento que se conectan completamente a las 784 neuronas de entrada, para poder establecer los pesos de estas relaciones se necesita una *matriz W* de pesos de 784 x 100;
adicionalmente cada neurona tiene un sesgo (i.e. ordenada en el origen) que representamos con un *array b* de 784 posiciones.

Vamos a inicializar las variables correspondientes. Los sesgos los inicializamos en ceros, los pesos los inicializamos aleatoriamente siguiendo una distribución normal con valores entre -1 y 1.

In [ ]:
tf.reset_default_graph()

In [ ]:
n0=784
n1=100
b = tf.Variable(tf.zeros((n1,)), name="sesgos")

In [ ]:
w = tf.Variable(tf.random_uniform((n0, n1), -1, 1), name="pesos")

Dispongo de 1000 registros de input, cada uno un vector de 784 datos:

Las variables en TF referencian tensores, y pueden entonces modificar sus valores. En sí, las variables de TF no tienen ningún estado, y no se puede entonces evaluar su valor hasta no haber sido inicializadas, así se haya especificado los valores iniciales que se desean.

In [ ]:
m=1000
X = tf.placeholder(tf.float32, (m, n0), name="inputs_X")

La capa de neuronas utiliza una función de activación sigmoide; podemos entonces calcular los valores de las activaciones:

In [ ]:
Z1 = tf.add(tf.matmul(X, w), b, "Z1")

In [ ]:
A1 = tf.nn.sigmoid(Z1, name="A1_sigmoide")

En este punto no se puede imprimir el valor de Z1, pues lo único que se ha hecho hasta ahora es definir el dataflow graph. Todavía no se ha calculado nada, ni se han inicializado los placeholders. Es necesario ejecutar el grafo en una sesión.

Veamos cómo queda el grafo en memoria. A bajo nivel, TensorFlow crea muchos mas nodos (operaciones) para poder soportar las definidas a alto nivel.

In [ ]:
tf.get_default_graph().get_operations()

Veamos el estado del grafo. Analicen lo que implicaría evaluar el nodo de inicialización.

In [ ]:
show_graph(tf.get_default_graph().as_graph_def())

Ejecutamos el grafo a través de una sesión; al no especificar ningun ambiente de ejecución TF utiliza el que esté configurado por defecto (CPU), pero se habría podido indicar explícitamente el ambiente de ejecución deseado.
Es necesario primero inicializar las variables en memoria, por lo que llamamos al método *global_variables.initializer*.

Le estamos mandando a la sesión como **fetch** el nodo de inicialización, la ejecución de esta operación es necesaria antes de poder ejecutar cualquier operación.

In [ ]:
sess = tf.Session()

In [ ]:
sess.run(tf.global_variables_initializer())

Como ya se inicializaron las variables, ya se puede evaluar cualquier operación. Es equivalente evaluar las operaciones con el método *eval* con una sesión dada a enviar la operación como operación de fetch a la sesión:

In [ ]:
b.eval(session=sess)

Vamos ahora a pedirle a la sesión que ejecute la operación del nodo A1, que incluimos como parte del argumento **fetch**. Para poder evaluar A1, necesitamos establecer los valores a asignar a los placeholders, por lo que creamos un array *valores* que asignaremos al *placeholder X* en un diccionario que se le envía a la sesión en el argumento **feed dict**.

In [ ]:
valores = np.random.random([m,n0])

In [ ]:
resultados = sess.run(A1, {X: valores})
resultados.shape

Obtenemos el resultado de los  registros de entrada (m=1000) para las neuronas de salida (n1=100).

In [ ]:
resultados[0]

También podíamos haber utilizado

In [ ]:
resultados = A1.eval(feed_dict={X: valores}, session=sess)

In [ ]:
resultados[0]

Se puede modificar las variables asignándoles valores después de haber sido inicializadas, con tal de respetar los rangos y dimensiones previamente establecidas al declararlas:

In [ ]:
sess.run(w.assign(tf.random_uniform((n0, n1), -0.5, 0.5)))

In [ ]:
show_graph(tf.get_default_graph().as_graph_def())

Para evitar problemas de recursos, siempre es aconsejable cerrar las sesiones:

In [ ]:
sess.close()

Se puede utilizar el comando *with* con una sesión para ponerse en el contexto de una sesión dada:

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    resultados = sess.run(A1, {X: valores})
resultados